In [1]:
import numpy as np
import pandas as pd
import mysql.connector as sql
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim as nm
from tqdm.notebook import tqdm

%matplotlib qt


In [2]:
#IMPORTING ALL THREE DATA FRAMES 
df1 = pd.read_csv('events_data.csv')
server = sql.connect(host='cpanel.insaid.co', database='Capstone1', user='student', password='student')

df2 = pd.read_sql('SELECT * FROM gender_age_train', con=server)
df3 = pd.read_sql('SELECT * FROM phone_brand_device_model', con=server)
server.close

#ADDITIONAL REQUIRED INFO
eng_names = np.array([['华为','Huawei'],
                      ['小米','Xiaomi'],
                      ['三星','Samsung'],
                      ['vivo','vivo'],
                      ['OPPO','OPPO'],
                      ['魅族','Meizu'],
                      ['酷派','Coolpad'],
                      ['乐视','LeEco'],
                      ['联想','Lenovo'],
                      ['HTC','HTC']])

In [3]:
# Check 1 - Whether longitude and latitude are always missing in pairs:
# This code creates two dataframes, containing the index values (row numbers) of the rows where either one is missing, and then checks if the two dataframes are identical.
# If they are identical, then they are always missing in pairs.
Check1 = df1[df1['latitude'].isnull()].index.equals(df1['longitude'].isnull().index)
print("Check 1 - Latitude and Longitude are always missing in pairs - " + str(Check1))

# Check 2 - Whether each city only exists in one state
# This code creates a dataframe containing unique city-state pairs, and the length of this dataframe is compared to the number of unique cities.
# If they are of different length, then we cannot calculate the state using the city
city_state_pairs = df1[df1['state'].notnull()][['city','state']].drop_duplicates()
unique_cities = df1['city'].nunique()
print('Check 2 - There are ' + str(len(city_state_pairs)) + ' unique city-state pairs, and ' + str(unique_cities) + ' unique cities')
#These two numbers are not equal. This means that some cities are present in multiple states. To verify:
city_state_pairs['city'].value_counts()[0:11]

# Check 3 - Whether we can use coordinates to find the state
Check3a = df1[df1['latitude'].isnull()].index.tolist()
Check3b = df1[df1['state'].isnull()].index.tolist()
for i in Check3a:
    if i in Check3b:
        print('Check 3 - In row ' + str(i) + ', both the coordinates and the city are missing')
# This prints nothing; therefore, we can use coordinates to find the city.
# This is foolproof, as each coordinate pair MUST refer to only one state

# Check 4 - Keep track of number of unique entries in "state"
# After adding the states using geopy, if the syntax is wrong, then this will help detect that:
print('Check 4 - There are ' + str(df1['state'].nunique()) + ' unique entries in the "state" column.')

Check 1 - Latitude and Longitude are always missing in pairs - False
Check 2 - There are 943 unique city-state pairs, and 933 unique cities
Check 4 - There are 32 unique entries in the "state" column.


In [4]:
#This is a list containing the index values of all rows where state is a null value.
null_records = df1[df1['state'].isnull()].index.tolist()
#This section of code fills in those values via geopy and a loop over the rows with null values
geolocator = nm(user_agent="my-applications")
for i in null_records:
    location = geolocator.reverse([df1.at[i,'latitude'],df1.at[i,'longitude']])
    # the code for location is taken from https://www.geeksforgeeks.org/python-remove-spaces-from-a-string/
    df1.at[i,'state'] = location.raw['address'].get('state', '').replace(' ','')

print(df1['state'].nunique())
df1 = df1[df1['state'] == "Maharashtra"]

32


In [6]:
df1.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 677168 entries, 9 to 3252946
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   event_id   677168 non-null  int64  
 1   device_id  677096 non-null  float64
 2   timestamp  677168 non-null  object 
 3   longitude  677105 non-null  float64
 4   latitude   677105 non-null  float64
 5   city       677168 non-null  object 
 6   state      677168 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 41.3+ MB


In [51]:
coords = df1[df1['latitude'].notnull()][['latitude','longitude']].reindex().values

In [56]:
plt.scatter(coords[0:,0],coords[0:,1],s=1)